In [1]:
import eyed3

import os
import subprocess
import re

from constants import ALBUM, ARTIST, SHORT_NAME, SOURCE_DIR, TOTAL_DISCS

In [37]:
def rename_wav_files(directory):
    pattern = re.compile(r'(\d{1,2})-(\d{1,2})\s.*\.wav$', re.IGNORECASE)

    for filename in os.listdir(directory):
        if filename.lower().endswith('.wav'):
            match = pattern.match(filename)
            if match:
                disc = int(match.group(1))
                track = int(match.group(2))
                new_filename = f"{SHORT_NAME} {disc:02d}-{track:02d}.wav"
                src = os.path.join(directory, filename)
                dst = os.path.join(directory, new_filename)
                print(f"Renaming '{filename}' -> '{new_filename}'")
                os.rename(src, dst)

def convert_from_wav_to_mp3(directory):
    for filename in os.listdir(directory):
        if filename.lower().endswith(".wav"):
            wav_path = os.path.join(directory, filename)
            base_name = os.path.splitext(filename)[0]
            mp3_name = base_name + ".mp3"
            mp3_path = os.path.join(directory, mp3_name)
            print(f"Converting {filename} -> {mp3_name}")
            subprocess.run([
                "ffmpeg", "-y", "-i", wav_path, "-codec:a", "libmp3lame", "-qscale:a", "2", mp3_path],
                check=True)

def delete_wav_files(directory):
    deleted_files = []

    for filename in os.listdir(directory):
        if filename.lower().endswith('.wav'):
            file_path = os.path.join(directory, filename)
            try:
                os.remove(file_path)
                deleted_files.append(filename)
                print(f"Deleted: {filename}")
            except Exception as e:
                print(f"Failed to delete {filename}: {e}")

    if not deleted_files:
        print("No .wav files found.")
    else:
        print(f"Deleted {len(deleted_files)} .wav file(s).")            

def add_tags(directory):
    pattern = re.compile(r'.*(\d{2})-(\d{2}).mp3$', re.IGNORECASE)
    for filename in os.listdir(directory):
        if filename.lower().endswith('.mp3'):
            base_name = os.path.splitext(filename)[0]
            mp3_path = os.path.join(directory, filename)
            match = pattern.match(filename)
            disc = int(match.group(1))
            track = int(match.group(2))
            audiofile = eyed3.load(mp3_path)
            if audiofile.tag is None:
                audiofile.initTag()
            audiofile.tag.title = base_name
            audiofile.tag.album = ALBUM
            audiofile.tag.artist = ARTIST
            audiofile.tag.disc_num = (disc, TOTAL_DISCS)
            audiofile.tag.track_num = track
            audiofile.tag.save()
            



In [38]:
#rename_wav_files(SOURCE_DIR)

In [39]:
#convert_from_wav_to_mp3(SOURCE_DIR)

In [40]:
#delete_wav_files(SOURCE_DIR)

In [41]:
add_tags(SOURCE_DIR)